# Simple CNN model for image classification

This notebook contains a simple CNN model to classify between faulty and normal transformers using their thermal images.

In [1]:
import torch
import torchvision
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torchinfo import summary

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
data_dir = "separated_data"
batch_size = 16
image_size = (256, 256)
num_epochs = 100

In [4]:
full_Dataset = torchvision.datasets.ImageFolder(
    root=data_dir,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize(image_size),
        torchvision.transforms.ToTensor(),
    ])
)

class_names = full_Dataset.classes

train_size = int(0.8 * len(full_Dataset))
val_size = len(full_Dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_Dataset, [train_size, val_size])


In [5]:
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

Training samples: 54
Validation samples: 14


In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Obtain an EfficientNet-B7 model pre-trained on ImageNet and fine-tune it on the transformer dataset.

In [7]:
model = torchvision.models.efficientnet_b7(weights='DEFAULT').to(device)
print(summary(model, input_size=(batch_size, 3, *image_size), col_names=["input_size", "output_size", "num_params", "trainable"]))

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [16, 3, 256, 256]         [16, 1000]                --                        True
├─Sequential: 1-1                                       [16, 3, 256, 256]         [16, 2560, 8, 8]          --                        True
│    └─Conv2dNormActivation: 2-1                        [16, 3, 256, 256]         [16, 64, 128, 128]        --                        True
│    │    └─Conv2d: 3-1                                 [16, 3, 256, 256]         [16, 64, 128, 128]        1,728                     True
│    │    └─BatchNorm2d: 3-2                            [16, 64, 128, 128]        [16, 64, 128, 128]        128                       True
│    │    └─SiLU: 3-3                                   [16, 64, 128, 128]        [16, 64, 128, 128]        --                        --
│    └─Sequential: 2-2  

Change the classifier layer to match the number of classes in the dataset.

In [8]:
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5, inplace=True),
    nn.Linear(model.classifier[1].in_features, len(class_names))
).to(device)
print(summary(model, input_size=(batch_size, 3, *image_size), col_names=["input_size", "output_size", "num_params", "trainable"]))

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [16, 3, 256, 256]         [16, 2]                   --                        True
├─Sequential: 1-1                                       [16, 3, 256, 256]         [16, 2560, 8, 8]          --                        True
│    └─Conv2dNormActivation: 2-1                        [16, 3, 256, 256]         [16, 64, 128, 128]        --                        True
│    │    └─Conv2d: 3-1                                 [16, 3, 256, 256]         [16, 64, 128, 128]        1,728                     True
│    │    └─BatchNorm2d: 3-2                            [16, 64, 128, 128]        [16, 64, 128, 128]        128                       True
│    │    └─SiLU: 3-3                                   [16, 64, 128, 128]        [16, 64, 128, 128]        --                        --
│    └─Sequential: 2-2  

Set only the classifier layer to be trainable, freezing the rest of the model.

In [9]:
for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

print(summary(model, input_size=(batch_size, 3, *image_size), col_names=["input_size", "output_size", "num_params", "trainable"]))

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [16, 3, 256, 256]         [16, 2]                   --                        Partial
├─Sequential: 1-1                                       [16, 3, 256, 256]         [16, 2560, 8, 8]          --                        False
│    └─Conv2dNormActivation: 2-1                        [16, 3, 256, 256]         [16, 64, 128, 128]        --                        False
│    │    └─Conv2d: 3-1                                 [16, 3, 256, 256]         [16, 64, 128, 128]        (1,728)                   False
│    │    └─BatchNorm2d: 3-2                            [16, 64, 128, 128]        [16, 64, 128, 128]        (128)                     False
│    │    └─SiLU: 3-3                                   [16, 64, 128, 128]        [16, 64, 128, 128]        --                        --
│    └─Sequential

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [11]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    model.eval()
    correct = 0
    total = 0
    with torch.inference_mode():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total

    if epoch % 10 == 0 or epoch == num_epochs - 1:
        print("=====================================================")
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}")
        print(f'Validation Accuracy: {accuracy:.2f}%')

Epoch [1/100], Training Loss: 0.7008
Validation Accuracy: 14.29%
Epoch [11/100], Training Loss: 0.6099
Validation Accuracy: 28.57%
Epoch [21/100], Training Loss: 0.5102
Validation Accuracy: 42.86%
Epoch [31/100], Training Loss: 0.4999
Validation Accuracy: 57.14%
Epoch [41/100], Training Loss: 0.4276
Validation Accuracy: 71.43%
Epoch [51/100], Training Loss: 0.4178
Validation Accuracy: 71.43%
Epoch [61/100], Training Loss: 0.4525
Validation Accuracy: 64.29%
Epoch [71/100], Training Loss: 0.4643
Validation Accuracy: 64.29%
Epoch [81/100], Training Loss: 0.3510
Validation Accuracy: 71.43%
Epoch [91/100], Training Loss: 0.3962
Validation Accuracy: 78.57%
Epoch [100/100], Training Loss: 0.3487
Validation Accuracy: 78.57%
